In [32]:
!pip install gradio together


In [28]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.0 MB/s eta 0:00:00


In [33]:
import os
import gradio as gr
from together import Together
import tempfile
from docx import Document

# Initialize the Together AI client with API key

TOGETHER_API_KEY = "TOGETHER_APT_KEY"
api_key = os.environ.get("TOGETHER_API_KEY", TOGETHER_API_KEY)

together_client = Together(api_key=api_key)

# Function to summarize code using LLM (Together.ai)
def summarize_code_with_llm(code: str) -> str:
    try:
        # Create the prompt to send to LLM
        prompt = f"""You are an expert software assistant. Your task is to analyze and summarize the provided code. Here's how to approach the task:

```
 {code}
        ```

        Explain what this code does, includes
1.  **Understand the Code**
   - List the main functions and classes.
   - Check which ones are used (skip unused/dead code).
   - Note if any function/class uses another (dependencies).

2.  **Summarize Each Important Function or Class**
   For each one:
   - What is its purpose?
   - What are its important parts?
   - How does it help the whole code?
   - What does it take as input and give as output?

3.  **Give a Quick Code Summary**
   - What does the whole file or module do overall?
   - How do the pieces work together?
   - Mention any important external libraries or tools.

 Use short bullet points. Avoid technical jargon.

Here is the code to summarize:
    - Give a quick summary of what the whole code does.
    - Explain how the main functions and classes work together to make it all work.
    - Mention any important tools or libraries the code is using.
"""

        # Get the completion from Together.ai
        response = together_client.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
            messages=[{"role": "user", "content": prompt}]
        )

        # Extract and return the summary
        return response.choices[0].message.content

    except Exception as e:
        return f"Error generating summary: {str(e)}"

# Function to create a DOCX file from the summary
def create_docx_from_summary(summary_text):
    doc = Document()
    doc.add_heading('Code Summary', 0)

    # Split by the "## Summary for" headers
    sections = summary_text.split("## Summary for ")

    for i, section in enumerate(sections):
        if i == 0 and not section.strip():
            continue  # Skip empty first section

        # Add file name as heading
        if i > 0:
            parts = section.split(":", 1)
            if len(parts) > 1:
                file_name = parts[0].strip()
                doc.add_heading(f"Summary for {file_name}", level=1)
                content = parts[1].strip()
            else:
                content = section.strip()

            # Add summary content with proper formatting
            for line in content.split("\n"):
                if line.strip():
                    doc.add_paragraph(line)

    # Save to a temporary file
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.docx')
    doc.save(temp_file.name)
    return temp_file.name

# Function to process files and summarize them
def process_files(files):
    if not files:
        return "Please upload at least one file to summarize.", None

    # Iterate over uploaded files and summarize each
    summaries = []

    for file in files:
        try:
            # Handle the file content correctly
            # In newer Gradio versions, file might be a file path
            if isinstance(file, str):
                with open(file, 'r', encoding='utf-8') as f:
                    code = f.read()
            else:
                # For older Gradio versions or different file object types
                # The file could be a UploadFile object with name and path attributes
                file_path = file.name
                with open(file_path, 'r', encoding='utf-8') as f:
                    code = f.read()

            # Get the summary of the code
            file_name = os.path.basename(file.name if hasattr(file, 'name') else file)
            summary = summarize_code_with_llm(code)
            summaries.append(f"## Summary for {file_name}:\n{summary}\n")
        except Exception as e:
            file_name = getattr(file, 'name', str(file))
            summaries.append(f"Error processing {file_name}: {str(e)}")

    summary_text = "\n".join(summaries)
    docx_path = create_docx_from_summary(summary_text)

    return summary_text, docx_path

# Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Code Summarizer")
    gr.Markdown("Upload JavaScript, TypeScript, or other code files to get an AI-generated summary of what the code does.")

    with gr.Row():
        # File input for multiple code files
        file_input = gr.File(
            file_count="multiple",
            label="Upload Files"
        )

    with gr.Row():
        # Button to trigger summarization
        submit_btn = gr.Button("Generate Summaries")

    with gr.Row():
        # Output text area to show the summaries
        summary_output = gr.Markdown(label="Summaries")

    with gr.Row():
        # File output for downloading the summary as DOCX
        docx_output = gr.File(label="Download Summary as DOCX")

    # Trigger the summarization process on button click
    submit_btn.click(fn=process_files, inputs=file_input, outputs=[summary_output, docx_output])

# Instructions for running the app
if __name__ == "__main__":
    print("Before running this app, make sure to:")
    print("1. Install required packages: pip install gradio together python-docx")
    print("2. Set your Together API key as an environment variable: export TOGETHER_API_KEY='your_api_key'")

    # Launch the Gradio interface
    demo.launch()

Before running this app, make sure to:
1. Install required packages: pip install gradio together python-docx
2. Set your Together API key as an environment variable: export TOGETHER_API_KEY='your_api_key'
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5cf9ec93c8cb430594.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
